séparer par type d'achat 

les clients achètent de manière générale du pgc et des PRODUITS FRAIS :
favoriser ce type d'achat

supprimer textile et bazar des calculs car il n'y a pas trop de recurrence c'est souvent des produits acheter aléatoirement au moment du besoin

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# Import libraries
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

train_dataframes = []
for i in tqdm(range(1, 2)):
    train_dataframes.append(pd.read_csv(f'data-train/train_data_part_{i}.csv'))
train_data = pd.concat(train_dataframes, ignore_index=True)

# free up memory by deleting the dataframes we no longer need
del train_dataframes

# This code reads the data from a CSV file named "products_data.csv" into a pandas DataFrame
products_data = pd.read_csv('data-train/products_data.csv')

# This code reads the data from a CSV file named "test_data.csv" into a pandas DataFrame
test_data = pd.read_csv('data-train/test_data.csv')

100%|██████████| 1/1 [00:20<00:00, 20.58s/it]
C:\Users\jeann\AppData\Local\Temp\ipykernel_13160\3163275204.py:15: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  products_data = pd.read_csv('data-train/products_data.csv')


In [3]:
columns_to_keep = ['date', 'transaction_id', 'customer_id', 'product_id']
train_data = train_data[columns_to_keep]

In [ ]:
import pandas as pd

# Assurer que la colonne 'date' est en format datetime
train_data['date'] = pd.to_datetime(train_data['date'])

# Extraire les informations temporelles
train_data['month'] = train_data['date'].dt.month        # Mois de l'année
train_data['day_of_week'] = train_data['date'].dt.weekday    # jour de la semaine (0 = lundi, 6 = dimanche)
l'information sur les jours de la semaine aurai pu etre interéssante mais pour les données test nous ne savons pas sur quel jour ce passe les achats 

# Afficher un aperçu des données
print(train_data.head())

# Suppression de la colonne date si elle n'est plus nécessaire pour éviter la multicolinéarité
#train_data = train_data.drop(columns=['date'])


        date       transaction_id   customer_id     product_id  month  \
0 2023-11-15  Transaction_1730125  Household_39   Product_5362     11   
1 2022-07-20  Transaction_1560535  Household_39  Product_67174      7   
2 2022-07-20  Transaction_1560535  Household_39  Product_82254      7   
3 2023-11-15  Transaction_1730125  Household_39   Product_3895     11   
4 2022-07-20  Transaction_1560535  Household_39  Product_34014      7   

   day_of_week  hour  
0            2     0  
1            2     0  
2            2     0  
3            2     0  
4            2     0  


In [6]:
# Trier les données par customer_id et date
train_data = train_data.sort_values(by=['customer_id', 'date'])

# Extraire les 10 premières transactions de chaque client
first_50_purchases = train_data.groupby('customer_id').head(50)

# Extraire des variables temporelles des dates
first_50_purchases['date'] = pd.to_datetime(first_50_purchases['date'])
first_50_purchases['month'] = first_50_purchases['date'].dt.month

# Vérifier les données
print(first_50_purchases.head())

# Optionnel : Suppression de la colonne date si non nécessaire
#first_10_purchases = first_10_purchases.drop(columns=['date'])


              date       transaction_id  customer_id     product_id  month  \
1256210 2022-03-01  Transaction_2515147  Household_1  Product_62373      3   
1521128 2022-03-01  Transaction_2515147  Household_1  Product_33145      3   
2048302 2022-03-01  Transaction_2515147  Household_1  Product_68212      3   
3378083 2022-03-01  Transaction_2515147  Household_1  Product_62388      3   
3378089 2022-03-01  Transaction_2515147  Household_1  Product_62136      3   

         day_of_week  hour  
1256210            1     0  
1521128            1     0  
2048302            1     0  
3378083            1     0  
3378089            1     0  


C:\Users\jeann\AppData\Local\Temp\ipykernel_13160\835597002.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_10_purchases['date'] = pd.to_datetime(first_10_purchases['date'])
C:\Users\jeann\AppData\Local\Temp\ipykernel_13160\835597002.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_10_purchases['month'] = first_10_purchases['date'].dt.month
